In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.options.display.max_columns = 50

In [3]:
def read_in_csv(file_path='./parking-geo.csv'):
    # let's be memory efficient when loading our data
    dtypes_dict = \
    {
        'ticket_number': np.int32,
        'violation_location': str,
        'license_plate_number': str,
        'license_plate_state': 'category',
        'license_plate_type': 'category',
        'zipcode': str,
        'violation_code': 'category',
        'violation_description': 'category',
        'unit': 'category',
        'unit_description': 'category',
        'vehicle_make': 'category',
        'fine_level1_amount': np.int32,
        'fine_level2_amount': np.int32,
        'current_amount_due': np.float64,
        'total_payments': np.float64,
        'ticket_queue': 'category',
        'notice_level': 'category',
        'hearing_disposition': 'category',
        'notice_number': np.int32,
        'dismissal_reason': str,
        'officer': str,
        'address': str,
        'license_hash': str,
        'year': np.int32,
        'month': 'category',
        'hour': 'category',
        'penalty': np.float64,
        'ward': 'category',
        'geocode_accuracy': np.float64,
        'geocode_accuracy_type': 'category',
        'geocoded_address': str,
        'geocoded_lng': str,
        'geocoded_lat': str,
        'geocoded_city': 'category',
        'geocoded_state': 'category'
    }
    #still better than strings
    parse_dates_list = \
    [
        'issue_date',
        'ticket_queue_date',
    ]
    
    # read csv into memory -- this takes quite a while
    df = pd.read_csv(file_path, dtype=dtypes_dict, parse_dates=parse_dates_list)
    return df

In [4]:
def calculate_summary_stats(df_raw, min_year = 1995, max_year = 2019):
    
    # private helper function
    def rank_series(series):
        out_series = series.rank(ascending=False)
        return out_series
    
    df_filtered = df_raw[
        (df_raw['year'] > min_year) & (df_raw['year'] < max_year) & 
        (df_raw['geocode_accuracy_type'].isin(['rooftop', 'range_interpolation', 'intersection', 'point'])) & 
        (df_raw['geocoded_city'] == 'Chicago')
    ]
    # not used
    # df_filtered_na = df_filtered[df_filtered['ward'].isnull()]
    df_filtered = df_filtered[df_filtered['ward'].notnull()]

    # calculate base dataframes
    df_dict = dict()
    df_dict['filtered'] = df_filtered
    df_dict['police_tickets'] = df_filtered[
        df_filtered['unit_description'].isin(['CPD', 'CPD-Other', 'CPD-Airport'])
    ]
    df_dict['contested_tickets'] = df_filtered[
        df_filtered['hearing_disposition'].isin(['Liable', 'Not Liable'])
    ]
    df_dict['paid_tickets'] = df_filtered[
        df_filtered['ticket_queue'] == 'Paid'
    ]
    df_dict['dismissed_tickets'] = df_filtered[
        df_filtered['ticket_queue'] == 'Dismissed'
    ]
    df_dict['seized_or_suspended_tickets'] = df_filtered[
        df_filtered['notice_level'].isin(['SEIZ', 'DLS'])
    ]
    df_dict['bankruptcy_tickets'] = df_filtered[
        df_filtered['ticket_queue'] == 'Bankruptcy'
    ]

    # group dataframes by ward
    gb_dict = dict()
    for key in df_dict:
        gb_dict[key] = df_dict[key].groupby('ward')

    # calculate the different stats
    out_dict = dict()
    ticket_count = gb_dict['filtered']['ticket_number'].count()
    out_dict['ticket_count'] = ticket_count
    out_dict['current_amount_due'] = gb_dict['filtered']['current_amount_due'].sum()
    out_dict['fine_level1_amount'] = gb_dict['filtered']['fine_level1_amount'].sum()
    out_dict['total_payments'] = gb_dict['filtered']['total_payments'].sum()
    out_dict['avg_per_ticket'] = out_dict['fine_level1_amount']/ticket_count
    out_dict['paid_pct'] = out_dict['total_payments']/(out_dict['current_amount_due']+out_dict['total_payments'])
    out_dict['police_ticket_count'] = gb_dict['police_tickets']['ticket_number'].count()
    out_dict['police_ticket_count_pct'] = out_dict['police_ticket_count'] / ticket_count
    out_dict['contested_ticket_count'] = gb_dict['contested_tickets']['ticket_number'].count()
    out_dict['contested_ticket_count_pct'] = out_dict['contested_ticket_count'] / ticket_count
    out_dict['paid_ticket_count'] = gb_dict['paid_tickets']['ticket_number'].count()
    out_dict['paid_ticket_count_pct'] = out_dict['paid_ticket_count'] / ticket_count
    out_dict['dismissed_ticket_count'] = gb_dict['dismissed_tickets']['ticket_number'].count()
    out_dict['dismissed_ticket_count_pct'] = out_dict['dismissed_ticket_count'] / ticket_count
    out_dict['seized_or_suspended_ticket_count'] = gb_dict['seized_or_suspended_tickets']['ticket_number'].count()
    out_dict['seized_or_suspended_ticket_count_pct'] = out_dict['seized_or_suspended_ticket_count'] / ticket_count
    out_dict['bankruptcy_ticket_count'] = gb_dict['bankruptcy_tickets']['ticket_number'].count()
    out_dict['bankruptcy_ticket_count_pct'] = out_dict['bankruptcy_ticket_count'] / ticket_count

    # calculate ranks; combine and format output dataframe
    df_out = pd.DataFrame()
    for key in out_dict:
        df_out[key] = out_dict[key]
        df_out[key+'_rank'] = rank_series(out_dict[key]).astype(int)
    df_out.index = df_out.index.astype(int)
    df_out = df_out.sort_index()
    
    return df_out

In [5]:
%%time
# takes quite a while...
df = read_in_csv()

CPU times: user 9min 52s, sys: 2min 43s, total: 12min 36s
Wall time: 13min 8s


In [6]:
%%time
df_1996to2018 = calculate_summary_stats(df, min_year=1995, max_year=2019)

CPU times: user 1min 26s, sys: 3min 1s, total: 4min 28s
Wall time: 5min 16s


In [7]:
%%time
df_2013to2017 = calculate_summary_stats(df, min_year=2012, max_year=2018)

CPU times: user 18.7 s, sys: 9.22 s, total: 27.9 s
Wall time: 21.7 s


In [8]:
df_1996to2018.to_csv('df_1996to2018.csv')
# df_1996to2018.to_csv()

In [9]:
df_2013to2017.to_csv('df_2013to2017.csv')
# df_2013to2017.to_csv()

In [10]:
df_check = pd.read_csv('./wardstotals.csv', index_col='ward').sort_index(ascending=True)
df_check5yr = pd.read_csv('./wardstotals5yr.csv', index_col='ward').sort_index(ascending=True)
df_1996to2018_check = df_check[df_1996to2018.columns.tolist()]
df_2013to2017_check = df_check5yr[df_2013to2017.columns.tolist()]

In [11]:
wardstotals_sql_minus_pandas = df_1996to2018_check - df_1996to2018
wardstotals5yr_sql_minus_pandas = df_2013to2017_check - df_2013to2017

In [12]:
df_1996to2018.head()

,ticket_count,ticket_count_rank,current_amount_due,current_amount_due_rank,fine_level1_amount,fine_level1_amount_rank,total_payments,total_payments_rank,avg_per_ticket,avg_per_ticket_rank,paid_pct,paid_pct_rank,police_ticket_count,police_ticket_count_rank,police_ticket_count_pct,police_ticket_count_pct_rank,contested_ticket_count,contested_ticket_count_rank,contested_ticket_count_pct,contested_ticket_count_pct_rank,paid_ticket_count,paid_ticket_count_rank,paid_ticket_count_pct,paid_ticket_count_pct_rank,dismissed_ticket_count,dismissed_ticket_count_rank,dismissed_ticket_count_pct,dismissed_ticket_count_pct_rank,seized_or_suspended_ticket_count,seized_or_suspended_ticket_count_rank,seized_or_suspended_ticket_count_pct,seized_or_suspended_ticket_count_pct_rank,bankruptcy_ticket_count,bankruptcy_ticket_count_rank,bankruptcy_ticket_count_pct,bankruptcy_ticket_count_pct_rank
ward,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1720717,8,3.940247e+07,16,100835215,8,9.224356e+07,6,58.600697,38,0.700694,7,598783,14,0.347985,48,120036,8,0.069759,35,1184804,6,0.688553,6,110457,8,0.064192,40,392455,7,0.228076,41,5904,25,0.003431,38
2,2081261,3,3.431393e+07,24,117298610,4,1.093965e+08,3,56.359395,46,0.761229,4,825312,5,0.396544,46,179269,4,0.086135,15,1482622,4,0.712367,5,175963,4,0.084546,13,388546,8,0.186688,47,7169,21,0.003445,37
3,1189111,12,4.315614e+07,12,73217160,10,6.189717e+07,12,61.573024,30,0.589198,29,764735,7,0.643115,10,104479,11,0.087863,10,705441,13,0.593251,28,102294,10,0.086026,11,405934,6,0.341376,15,16634,13,0.013989,16
4,1783330,6,4.619477e+07,9,101895980,7,9.088336e+07,7,57.138040,45,0.663004,14,1054462,2,0.591288,20,171359,5,0.096089,5,1121035,8,0.628619,19,179054,3,0.100404,2,510207,4,0.286098,26,16886,12,0.009469,20
5,1082520,16,3.955258e+07,15,68784395,13,5.915885e+07,14,63.540992,24,0.599311,25,491319,23,0.453866,39,89974,15,0.083115,21,652664,18,0.602912,26,93870,13,0.086714,10,369375,13,0.341218,16,17008,11,0.015711,15


In [13]:
wardstotals_sql_minus_pandas

,ticket_count,ticket_count_rank,current_amount_due,current_amount_due_rank,fine_level1_amount,fine_level1_amount_rank,total_payments,total_payments_rank,avg_per_ticket,avg_per_ticket_rank,paid_pct,paid_pct_rank,police_ticket_count,police_ticket_count_rank,police_ticket_count_pct,police_ticket_count_pct_rank,contested_ticket_count,contested_ticket_count_rank,contested_ticket_count_pct,contested_ticket_count_pct_rank,paid_ticket_count,paid_ticket_count_rank,paid_ticket_count_pct,paid_ticket_count_pct_rank,dismissed_ticket_count,dismissed_ticket_count_rank,dismissed_ticket_count_pct,dismissed_ticket_count_pct_rank,seized_or_suspended_ticket_count,seized_or_suspended_ticket_count_rank,seized_or_suspended_ticket_count_pct,seized_or_suspended_ticket_count_pct_rank,bankruptcy_ticket_count,bankruptcy_ticket_count_rank,bankruptcy_ticket_count_pct,bankruptcy_ticket_count_pct_rank
ward,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,2.923608e-05,0,0,0,-1.730025e-05,0,0.000000e+00,0,-1.950662e-13,0,0,0,0.000000e+00,0,0,0,0.000000e+00,0,0,0,0.000000e+00,0,0,0,-1.387779e-17,0,0,0,-2.775558e-17,0,0,0,0.000000e+00,0
2,0,0,1.396239e-05,0,0,0,-1.676381e-05,0,-3.552714e-14,0,-1.012523e-13,0,0,0,0.000000e+00,0,0,0,-1.387779e-17,0,0,0,0.000000e+00,0,0,0,-1.387779e-17,0,0,1,0.000000e+00,0,0,0,4.336809e-19,0
3,21,0,1.334800e+03,0,2755,0,1.791000e+03,0,1.229440e-03,0,-4.827330e-07,0,19,0,4.620670e-06,0,3,0,9.711912e-07,0,8,0,-3.749177e-06,0,0,0,-1.519207e-06,0,7,0,-1.420337e-07,0,0,0,-2.470379e-07,0
4,1935,0,7.209669e+04,0,109620,0,8.887544e+04,0,-5.277124e-04,0,-1.300631e-04,0,1512,0,2.060519e-04,0,82,0,-5.821705e-05,0,1071,0,-8.143196e-05,0,115,0,-4.440927e-05,0,652,0,5.511816e-05,0,15,0,-1.860863e-06,0
5,309,0,1.934395e+04,0,22740,0,1.616363e+04,0,2.868258e-03,0,-5.181380e-05,0,262,0,1.124420e-04,0,24,0,-1.553925e-06,0,146,0,-3.721707e-05,0,26,0,-7.339376e-07,0,141,0,3.284334e-05,0,7,0,1.981061e-06,0
6,0,0,3.892928e-05,0,0,0,1.294911e-05,0,-2.842171e-14,0,-1.255662e-13,0,0,0,0.000000e+00,0,0,0,0.000000e+00,0,0,0,0.000000e+00,0,0,0,0.000000e+00,0,0,0,-5.551115e-17,0,0,0,3.469447e-18,0
7,8,0,3.432000e+02,0,435,0,1.720000e+02,0,-2.007704e-04,0,-9.814407e-07,0,6,0,2.792238e-06,0,0,0,-1.041613e-06,0,3,0,-1.347219e-06,0,1,0,5.219463e-07,0,4,0,3.939448e-07,0,0,0,-3.241942e-07,0
8,395,0,2.151348e+04,0,24900,0,1.981637e+04,0,-3.733468e-03,0,-1.549615e-05,0,310,0,1.642837e-04,0,26,0,-1.101439e-05,0,210,0,4.370182e-06,0,29,0,-9.171336e-06,0,181,0,1.481744e-05,0,11,0,2.944374e-06,0
9,39,0,2.016200e+03,0,2145,0,1.295180e+03,0,-1.554128e-03,0,-6.946400e-06,0,20,0,-8.856891e-06,0,3,0,-6.244180e-07,0,16,0,-9.113263e-06,0,3,0,-6.428599e-07,0,16,0,-1.610630e-06,0,0,0,-2.085649e-06,0


In [14]:
wardstotals5yr_sql_minus_pandas

,ticket_count,ticket_count_rank,current_amount_due,current_amount_due_rank,fine_level1_amount,fine_level1_amount_rank,total_payments,total_payments_rank,avg_per_ticket,avg_per_ticket_rank,paid_pct,paid_pct_rank,police_ticket_count,police_ticket_count_rank,police_ticket_count_pct,police_ticket_count_pct_rank,contested_ticket_count,contested_ticket_count_rank,contested_ticket_count_pct,contested_ticket_count_pct_rank,paid_ticket_count,paid_ticket_count_rank,paid_ticket_count_pct,paid_ticket_count_pct_rank,dismissed_ticket_count,dismissed_ticket_count_rank,dismissed_ticket_count_pct,dismissed_ticket_count_pct_rank,seized_or_suspended_ticket_count,seized_or_suspended_ticket_count_rank,seized_or_suspended_ticket_count_pct,seized_or_suspended_ticket_count_pct_rank,bankruptcy_ticket_count,bankruptcy_ticket_count_rank,bankruptcy_ticket_count_pct,bankruptcy_ticket_count_pct_rank
ward,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,-1.899898e-06,0,0,0,3.993511e-06,0,4.263256e-14,0,6.339373e-14,0,0,0,0.000000e+00,0,0,0,-1.387779e-17,0,0,0,1.110223e-16,0,0,0,0.000000e+00,0,0,0,0.000000e+00,0,0,0,-4.336809e-19,0
2,0,0,3.501773e-07,0,0,0,-2.421439e-07,0,0.000000e+00,0,-8.326673e-15,0,0,0,5.551115e-17,0,0,0,0.000000e+00,0,0,0,1.110223e-16,0,0,0,0.000000e+00,0,0,0,0.000000e+00,0,0,0,0.000000e+00,0
3,0,0,-4.464760e-06,0,0,0,-4.358590e-07,0,4.263256e-14,0,6.961098e-14,0,0,0,1.110223e-16,0,0,0,-4.163336e-17,0,0,0,-1.110223e-16,0,0,0,1.387779e-17,0,0,0,-5.551115e-17,0,0,0,0.000000e+00,0
4,191,0,1.157456e+04,0,17630,0,1.091980e+04,0,8.052898e-03,0,-1.014012e-04,0,169,0,3.577633e-04,0,25,0,7.464996e-06,0,100,0,-8.651039e-05,0,24,0,1.940404e-05,0,55,0,4.309093e-05,0,3,0,1.350616e-06,0
5,74,0,5.782750e+03,0,6700,0,5.087310e+03,0,1.476387e-03,0,-2.161579e-05,0,65,0,1.516915e-04,0,8,0,-1.613823e-06,0,38,0,-2.606575e-05,0,7,0,2.165708e-06,1,30,0,3.373137e-05,0,2,0,9.448280e-07,0
6,0,0,2.644956e-06,0,0,0,-1.838431e-06,0,-5.684342e-14,0,-6.306067e-14,0,0,0,-1.110223e-16,0,0,0,-1.387779e-17,0,0,0,5.551115e-17,0,0,0,0.000000e+00,0,0,0,-5.551115e-17,0,0,0,0.000000e+00,0
7,0,0,-5.258247e-06,0,0,0,-1.097098e-06,0,0.000000e+00,0,4.674039e-14,0,0,0,-1.110223e-16,0,0,0,-1.387779e-17,0,0,0,-1.110223e-16,0,0,0,0.000000e+00,0,0,0,-5.551115e-17,0,0,0,-1.387779e-17,0
8,69,0,9.443920e+03,0,7150,0,3.961200e+03,0,2.956866e-03,0,-4.897767e-05,0,45,0,2.201327e-05,0,4,0,-2.718727e-05,0,29,0,-1.163874e-05,0,6,0,-4.036440e-06,0,32,0,2.176421e-05,0,5,0,1.523846e-05,0
9,3,0,1.788000e+02,0,180,0,8.518000e+01,0,-1.021634e-03,0,-8.265678e-07,0,1,0,-9.803720e-06,0,0,0,-3.072081e-06,0,1,0,-2.817362e-06,0,0,0,-2.497483e-06,0,1,0,-2.420732e-06,0,0,0,-1.046434e-06,0


In [15]:
df_compare = pd.DataFrame()
df_compare['5yr_ticket_count_identical'] = ~wardstotals5yr_sql_minus_pandas['ticket_count'].astype(bool)
df_compare['all_yr_ticket_count_identical'] = ~wardstotals_sql_minus_pandas['ticket_count'].astype(bool)
df_compare

,5yr_ticket_count_identical,all_yr_ticket_count_identical
ward,,
1,True,True
2,True,True
3,True,False
4,False,False
5,False,False
6,True,True
7,True,False
8,False,False
9,False,False


In [16]:
# df_full_compare = pd.DataFrame()
# columns = df_1996to2018.columns.tolist()
# for column in columns:
    

In [17]:
wardstotals5yr_sql_minus_pandas.to_csv('./wardstotals5yr_sql_minus_pandas.csv')
wardstotals_sql_minus_pandas.to_csv('./wardstotals_sql_minus_pandas.csv')
df_compare.to_csv('./ticket_count_identical.csv')

In [18]:
%%time
# check out ward 7
df_filtered_check = df[
    (df['year'] > 1995) & (df['year'] < 2019) & 
    (df['geocode_accuracy_type'].isin(['rooftop', 'range_interpolation', 'intersection', 'point'])) & 
    (df['geocoded_city'] == 'Chicago')
]
df_filtered_check = df_filtered_check[df_filtered_check['ward'].notnull()]

KeyboardInterrupt: 

In [19]:
((df['geocoded_city'] != 'Chicago') & (df['ward'].notnull())).sum()

679371